In [1]:
from crewai import Agent, Task, Crew
import os
from dotenv import load_dotenv
import openai
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

In [2]:
# Carregar variáveis do .env
load_dotenv()

True

## Teste de multi-agentes guardando prompts anteriores

In [ ]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew

load_dotenv()

# 1) O codigo tem dois agentes: COMPRADOR e VENDEDOR
comprador = Agent(
    role="Comprador de Camisas",
    goal="Comprar uma camisa branca para um casamento. Já tem o terno, só precisa da camisa.",
    backstory=(
        "Você é um cliente que quer economizar; procura algo bonito e não muito caro. "
        "Fale de forma breve, pergunte preços, negocie se possível."
    ),
    memory=False,
    verbose=False,
    allow_delegation=False,
    llm=os.getenv("OPENAI_MODEL_NAME") or "gpt-3.5-turbo"
)

vendedor = Agent(
    role="Vendedor de Loja de Roupas",
    goal="Vender a melhor camisa branca possível para o cliente.",
    backstory=(
        "Você conhece bem o estoque. Procure oferecer 3 opções, discuta preços e tente concretizar a venda."
    ),
    memory=False,
    verbose=False,
    allow_delegation=False,
    llm=os.getenv("OPENAI_MODEL_NAME") or "gpt-3.5-turbo"
)

# 2) O codigo vai manter um histórico de conversas, como texto puro
#    simulando um diálogo estilo:
#      "Comprador: ...\nVendedor: ...\nComprador: ...\n" assim os agentes quando recebem o histórico sabem quem falou o que
historico = """(Início da conversa)
Comprador: Olá, tudo bem? Preciso de uma camisa branca para um casamento.
"""

def gerar_proxima_fala(agent: Agent, conversa: str, quem_fala: str) -> str:
    """
    Cria uma Task para o 'agent' gerar a próxima fala no diálogo,
    recebendo como contexto todo o 'conversa' (histórico) e
    instruindo para falar como 'quem_fala'.
    Retorna apenas o texto da fala (sem prefixos).
    """
    # Monta o prompt de forma simples:
    prompt = f"""
Este é o histórico da conversa até agora:
----------------
{conversa}
----------------

Agora é a vez de {quem_fala} falar.
Responda como se você fosse {quem_fala} no diálogo.
Não faça rodeios sobre a própria IA, apenas continue a conversa.
"""

    # Cria a Task
    t = Task(
        description=prompt,
        expected_output=f"A próxima fala do {quem_fala} no diálogo.",
        agent=agent
    )
    # Executa via um Crew temporário
    c = Crew(agents=[agent], tasks=[t], verbose=False)
    result = c.kickoff()
    return result.raw.strip()


def executar_dialogo():
    # Para demonstrar, vao ser feitas 6 trocas de falas (3 rodadas). 3 rodadas, cada 2 falas => total 6 falas
    global historico

    # Já começa com o comprador na conversa (definido acima).
    # Vamos alternar: Vendedor -> Comprador -> Vendedor -> Comprador ...
    for rodada in range(3):
        # 1) Vendedor fala
        fala_vendedor = gerar_proxima_fala(vendedor, historico, "Vendedor")
        historico += f"\nVendedor: {fala_vendedor}\n"

        # 2) Comprador fala
        fala_comprador = gerar_proxima_fala(comprador, historico, "Comprador")
        historico += f"\nComprador: {fala_comprador}\n"

    # imprimimos todo o diálogo
    print("=== Diálogo Gerado ===")
    print(historico)

if __name__ == "__main__":
    executar_dialogo()


Overriding of current TracerProvider is not allowed


Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed


=== Diálogo Gerado ===
(Início da conversa)
Comprador: Olá, tudo bem? Preciso de uma camisa branca para um casamento.

Vendedor: Olá! Fico feliz em te ajudar a encontrar a camisa branca perfeita para o casamento. Tenho três ótimas opções que podem interessar:

1. **Camisa de Algodão Premium** - Esta camisa é feita 100% em algodão egípcio, com um toque macio e um caimento excelente. É ideal para ocasiões formais. O preço é R$ 199,90.

2. **Camisa Slim Fit** - Com um design moderno e corte ajustado, essa camisa é perfeita para quem busca um visual elegante e contemporâneo. O tecido é leve e respirável, ideal para o clima do casamento. Essa custa R$ 169,90.

3. **Camisa Branca Clássica com Detalhes** - Esta opção traz um toque especial com detalhes sutis nas mangas e colarinho, sem perder a sofisticação. É um pouco mais casual, mas ainda assim elegante. O preço é R$ 149,90.

Você gostaria de experimentar alguma dessas opções? Posso te ajudar a encontrar o tamanho certo!

Comprador: Olá! A

## Teste multi-agentes 2 com memory

In [ ]:
# Carregar variáveis do .env
load_dotenv()

# Usar as variáveis carregadas
api_key = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("OPENAI_MODEL_NAME", "gpt-4-turbo")  # Define um padrão caso não esteja no .env

# Configuração do modelo de IA
llm = ChatOpenAI(model_name=model_name, openai_api_key=api_key, temperature=0.3)

# Memória resumida para manter o contexto sem gastar muitos tokens
memory = ConversationSummaryMemory(llm=llm)

# Definição dos agentes
comprador = Agent(
    role="Comprador",
    goal="Conversar com o vendedor e negociar diretamente o melhor preço para uma camisa branca de casamento.",
    backstory="Você tem um casamento neste fim de semana e precisa de uma camisa branca elegante. "
              "Você quer pagar um preço justo e está pronto para negociar diretamente.",
    llm=llm,
    memory=memory
)

vendedor = Agent(
    role="Vendedor de Loja de Roupas",
    goal="Interagir com o comprador de maneira dinâmica e negociar diretamente o preço de uma camisa branca.",
    backstory="Você trabalha em uma loja de roupas sofisticada e precisa vender camisas brancas. "
              "Você responde diretamente ao comprador e apresenta os melhores argumentos para justificar o preço.",
    llm=llm,
    memory=memory
)

# Definição da tarefa principal (negociação de compra)
task = Task(
    description=(
        "O comprador e o vendedor devem conversar diretamente, alternando suas falas, para negociar a compra "
        "de uma camisa branca para um casamento. O vendedor apresenta os produtos e o comprador negocia o preço. "
        "A conversa deve ser um diálogo interativo e não uma narração."
    ),
    agent=comprador,  # Define um agente principal
    memory=memory,
    expected_output="Um diálogo interativo concluindo a negociação com um acordo entre os dois agentes."
)

# Criar a equipe com os dois agentes
crew = Crew(
    agents=[comprador, vendedor],
    tasks=[task]
)

# Iniciar a conversa
resultado = crew.kickoff()
print(resultado)

# Converter o resultado para string antes de salvar
resultado_str = str(resultado)

# Exportar conversa para um arquivo
with open("conversa.txt", "w", encoding="utf-8") as file:
    file.write(resultado_str)

Overriding of current TracerProvider is not allowed


**Comprador:** Olá! Estou procurando uma camisa branca elegante para um casamento neste fim de semana. Você tem alguma opção disponível?  

**Vendedor:** Olá! Sim, temos várias camisas brancas. Esta aqui é uma camisa de algodão de alta qualidade, perfeita para ocasiões formais. Ela custa R$ 250.  

**Comprador:** Ela parece ótima, mas o preço está um pouco acima do que eu esperava. Você poderia fazer um desconto?  

**Vendedor:** Entendo, mas essa camisa é realmente de alta qualidade. Posso oferecer um desconto de 10%, o que a deixaria por R$ 225.  

**Comprador:** Agradeço a oferta, mas ainda assim está um pouco acima do meu orçamento. Se eu pagar em dinheiro, você poderia reduzir um pouco mais?  

**Vendedor:** Se você pagar em dinheiro, posso baixar para R$ 210. É o melhor que posso fazer.  

**Comprador:** R$ 210 é um preço mais justo, mas eu estava pensando em algo em torno de R$ 200. Você conseguiria fazer isso?  

**Vendedor:** Hmmm, R$ 200 é um pouco apertado, mas eu realmente 

In [ ]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory

def gerar_resposta(role, goal, memory, llm, mensagem_outro=None):
    """
    Gera a resposta de um "agente" (sem classe) baseado no resumo da conversa
    e na última mensagem do outro agente.
    """
    # 1. Obtemos o histórico resumido armazenado na memória.
    contexto = memory.load_memory_variables({})["history"]

    # 2. Montamos o prompt para o LLM.
    prompt = (
        f"Você é o {role}.\n"
        f"Seu objetivo: {goal}\n\n"
        f"Contexto resumido da conversa até agora:\n"
        f"{contexto}\n\n"
    )
    if mensagem_outro:
        prompt += f"O outro interlocutor disse: {mensagem_outro}\n"
    prompt += "Responda agora:\n"

    # 3. Gera a resposta usando o LLM.
    resposta = llm.predict(prompt)

    # 4. Salvamos o turno (input/output) na memória:
    #    "input": o que o outro agente falou,
    #    "output": a resposta deste agente.
    memory.save_context(
        {"input": mensagem_outro if mensagem_outro else ""},
        {"output": resposta}
    )

    return resposta


def main():
    # Carrega variáveis do .env (ou configure manualmente)
    api_key = os.getenv("OPENAI_API_KEY")
    model_name = os.getenv("OPENAI_MODEL_NAME", "gpt-4")

    # Configura LLM
    llm = ChatOpenAI(
        model_name=model_name,
        openai_api_key=api_key,
        temperature=0.3
    )

    # Cria memória resumida
    memory = ConversationSummaryMemory(llm=llm)

    # Definimos os dados dos "agentes" em dicionários:
    comprador = {
        "role": "Comprador",
        "goal": "Conversar com o vendedor e negociar diretamente o melhor preço para uma camisa branca de casamento."
    }
    vendedor = {
        "role": "Vendedor",
        "goal": "Vender a camisa branca e justificar o preço."
    }

    # Vamos fazer 6 trocas de mensagem (3 rodadas de cada lado).
    num_trocas = 6
    mensagem_atual = None

    for i in range(num_trocas):
        # Decide se é a vez do comprador ou do vendedor:
        if i % 2 == 0:  
            # Vez do comprador
            role = comprador["role"]
            goal = comprador["goal"]
        else:
            # Vez do vendedor
            role = vendedor["role"]
            goal = vendedor["goal"]

        # Gera resposta para o turno atual
        resposta = gerar_resposta(role, goal, memory, llm, mensagem_atual)
        
        # Imprime e prepara para o próximo turno
        print(f"{role}: {resposta}\n")
        mensagem_atual = resposta

    # Ao final, se quiser salvar o resumo (ou a conversa final) em um arquivo:
    resumo_final = memory.load_memory_variables({})["history"]
    with open("conversa.txt", "w", encoding="utf-8") as f:
        f.write(resumo_final)


if __name__ == "__main__":
    main()
